In [1]:
import csv
import re

import numpy as np
import pandas as pd

import sklearn
import matplotlib.pyplot as plt
# import eli5

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import zero_one_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

from sklearn.metrics import SCORERS
# import wehs_helpers as wh

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
bosnian_df = pd.read_csv('bosnian.csv',delimiter=';',encoding="utf8")

In [ ]:
bosnian_df[:5]

In [ ]:
bosnian_df_transpon=bosnian_df.transpose()

In [ ]:
bosnian_df_transpon[:]

In [ ]:
# bosnian_df_transpon['index'] = bosnian_df_transpon.index
bosnian_df_transpon2=bosnian_df_transpon.reset_index()
bosnian_df_transpon2

In [ ]:
bosnian_df_transpon2=bosnian_df_transpon2.rename(columns={"index": "q", 0: "a1", 1: "a2", 2: "a3", 3: "a4", 4: "a5", 5: "a6"})

In [ ]:
print(list(bosnian_df_transpon2.columns))

In [ ]:
bosnian_df_transpon2

In [ ]:
qdf=pd.DataFrame(bosnian_df_transpon2["q"]).copy()
print(list(qdf.columns))
qdf=qdf.rename(columns={"q": "text_input"})
qdf.loc[:, 'text_label'] = 'question'+qdf.index.astype(str)
qdf

In [ ]:
a1df=pd.DataFrame(bosnian_df_transpon2["a1"]).copy()
print(list(a1df.columns))
a1df=a1df.rename(columns={"a1": "text_input"})
a1df.loc[:, 'text_label'] = 'question'+qdf.index.astype(str)+'_answer1'
a1df

In [ ]:
a2df=pd.DataFrame(bosnian_df_transpon2["a2"]).copy()
a2df=a2df.rename(columns={"a2": "text_input"})
a2df.loc[:, 'text_label'] = 'question'+qdf.index.astype(str)+'_answer2'

a3df=pd.DataFrame(bosnian_df_transpon2["a3"]).copy()
a3df=a3df.rename(columns={"a3": "text_input"})
a3df.loc[:, 'text_label'] = 'question'+qdf.index.astype(str)+'_answer3'

a4df=pd.DataFrame(bosnian_df_transpon2["a4"]).copy()
a4df=a4df.rename(columns={"a4": "text_input"})
a4df.loc[:, 'text_label'] = 'question'+qdf.index.astype(str)+'_answer4'

a5df=pd.DataFrame(bosnian_df_transpon2["a5"]).copy()
a5df=a5df.rename(columns={"a5": "text_input"})
a5df.loc[:, 'text_label'] = 'question'+qdf.index.astype(str)+'_answer5'

a6df=pd.DataFrame(bosnian_df_transpon2["a6"]).copy()
a6df=a6df.rename(columns={"a6": "text_input"})
a6df.loc[:, 'text_label'] = 'question'+qdf.index.astype(str)+'_answer6'


In [ ]:
to_translate_df = pd.concat([qdf,a1df,a2df,a3df,a4df,a5df,a6df], ignore_index=True)
to_translate_df[:50]

In [ ]:
# !pip install nlpcloud